# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [ ]:
!pip install selenium

In [25]:
#passo a passo
#1: Pegar a cotação do Euro
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


#usamos para abrir o navegador
navegador = webdriver.Chrome()

navegador.get('https://www.google.com/webhp?hl=en&sa=X&ved=0ahUKEwifpp3J9Mr6AhXhO7kGHVHCC_YQPAgI')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Euro')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)


#2: Pegar a cotação do Dólar
navegador.get('https://www.google.com/webhp?hl=en&sa=X&ved=0ahUKEwifpp3J9Mr6AhXhO7kGHVHCC_YQPAgI')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Dólar')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dollar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dollar)


#3: Pegar a cotação do Ouro
navegador.get('https://dolarhoje.com/ouro-hoje/')
cotacao_ouro = navegador.find_element('xpath','//*[@id="nacional"]').get_attribute('value')
#aqui trocamos um caractere por outro
cotacao_ouro = cotacao_ouro.replace(',','.')
print(cotacao_ouro)

#usamos para sair do navegador
navegador.quit()


#4: Importar a lista de produtos
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)


#5: Recalcular o preço de cada produto
# atualizar a cotação
# nas linhas onde na coluna "Moeda" = Dólar
# se eu quisesse editar somente a linha, poderia fazer assim
#tabela.loc[0,"Cotação"] = float(cotacao_dollar) aonde eu subistituiria o valor em cotação pelo dollar (mas somente em uma)
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dollar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)


# atualizamos a coluna Preço de Compra fazendo este cálculo aqui (preço base original * cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizamos o coluna Preço de Venda fazendo esse cálculo aqui (preço base reais * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

# tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format) caso queira colocar em dinheiro e casas decimais

display(tabela)


#6: Salvar os novos preços dos produtos *exportar a tabela*
#index=False serve para ser exportado sem aqueles numeros de base
tabela.to_excel('Produtos Novo.xlsx', index=False)

5.1483
5.1967
286.47


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.1967,5196.648033,1.40,7275.307246
1,Carro Renault,4500.00,Euro,5.1483,23167.350000,2.00,46334.700000
2,Notebook Dell,899.99,Dólar,5.1967,4676.978033,1.70,7950.862656
3,IPhone,799.00,Dólar,5.1967,4152.163300,1.70,7058.677610
4,Carro Fiat,3000.00,Euro,5.1483,15444.900000,1.90,29345.310000
5,Celular Xiaomi,480.48,Dólar,5.1967,2496.910416,2.00,4993.820832
6,Joia 20g,20.00,Ouro,286.4700,5729.400000,1.15,6588.810000
